In [ ]:
#|hide
#|eval: false
! [ -e /content ] && pip install -Uqq fastrl['dev'] pyvirtualdisplay && \
                     apt-get install -y xvfb python-opengl > /dev/null 2>&1 
# NOTE: IF YOU SEE VERSION ERRORS, IT IS SAFE TO IGNORE THEM. COLAB IS BEHIND IN SOME OF THE PACKAGE VERSIONS

In [ ]:
#|hide
#|eval: false
from fastcore.imports import in_colab
# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab():
    from nbdev.showdoc import *
    from nbdev.imports import *
    if not os.environ.get("IN_TEST", None):
        assert IN_NOTEBOOK
        assert not IN_COLAB
        assert IN_IPYTHON
else:
    # Virutual display is needed for colab
    from pyvirtualdisplay import Display
    display = Display(visible=0, size=(400, 300))
    display.start()

In [ ]:
#|default_exp agents.ddpg

In [ ]:
#|export
# Python native modules
import os
from typing import *
# Third party libs
from fastcore.all import *
import torchdata.datapipes as dp
from torch import nn
import torch
from  torchdata.dataloader2.graph import DataPipe
# Local modules
from fastrl.torch_core import *
from fastrl.pipes.core import *
from fastrl.data.block import *
from fastrl.data.dataloader2 import *
from fastrl.agents.core import *
from fastrl.learner.core import *
from fastrl.loggers.core import *

# DDPG 
> Deep Deterministic Policy Gradiant for continuous action domains

[(Lillicrap et al., 2016) [DDPG] Continuous Control with Deep Reinforcement Learning](https://arxiv.org/abs/1509.02971) based on the 
DPG algorithm in [(Silver et al., 2014) [DPG] Deterministic Policy Gradient Algorithms](http://proceedings.mlr.press/v32/silver14.pdf).

DDPG uses an actor-critic architecture and has a similar training / learning paradym to DQNs.

## Model

In [ ]:
#|export
class Critic(Module):
    def __init__(
            self,
            state_sz:int,  # The input dim of the state
            action_sz:int, # The input dim of the actions
            hidden=512,    # Number of neurons connected between the 2 input/output layers
            head_layer:Module=nn.Linear, # Output layer
            activition_fn:Module=nn.ReLU # The activation function
        ):
        # TODO: Add batch normalization
        self.layers = nn.Sequential(
            nn.Linear(state_sz+action_sz,hidden),
            activition_fn(),
            head_layer(hidden,1),
        )
    def forward(
            self,
            x:torch.Tensor # A single tensor of shape [Batch,`state_sz`+`action_sz`]
            # A single tensor of shape [B,1] representing the cumulative value estimate of state+action combinations  
        ) -> torch.Tensor: 
        return self.layers(x)

add_docs(
Critic,
"Takes a single tensor of size [B,`state_sz`+`action_sz`] -> [B,1] outputs a 1d tensor repersenting the value",
forward="""Takes in a single tensor of a state tensor and action tensor and output
 the culative value estimates of that state,action combination"""
)

The `Critic` is used by `DDPG` to estimate the state-action pairs and is updated using the 
the Bellman-Equation similarly to DQN/Q-Learning and is represeted by $Q(s,a)$

In [ ]:
torch.manual_seed(0)
critic = Critic(4,2)

state = torch.randn(1,4)
action = torch.randn(1,2)

with torch.no_grad():
    test_eq(
        str(critic(torch.cat((state,action),dim=1))),
        str(tensor([[0.0040]]))
    )

In [ ]:
#|export
class Actor(Module):
    def __init__(self,
                 state_sz:int,  # The input dim of the state
                 action_sz:int, # The output dim of the actions
                 hidden=512,    # Number of neurons connected between the 2 input/output layers
                 head_layer:Module=nn.Linear, # Output layer
                 activition_fn:Module=nn.ReLU # The activiation function
                ):
        # TODO: Add batch normalization
        self.layers = nn.Sequential(
            nn.Linear(state_sz,hidden),
            activition_fn(),
            head_layer(hidden,action_sz),
        )
    def forward(self,x): return self.layers(x)

The `Actor` is used by `DDPG` to predict actions based on state inputs and is represeted by $\mu(s|\theta^\mu)$

In [ ]:
torch.manual_seed(0)
actor = Actor(4,2)

state = torch.randn(1,4)

with torch.no_grad():
    test_eq(
        str(actor(state)),
        str(tensor([[ 0.1440, -0.3434]]))
    )

## Agent

In [ ]:
#|export
def DDPGAgent(
    model,
    logger_bases=None,
    dp_augmentation_fns:Optional[List[DataPipeAugmentationFn]]=None
)->AgentHead:
    agent_base = AgentBase(model,logger_bases=ifnone(logger_bases,[CacheLoggerBase()]))
    agent = StepFieldSelector(agent_base,field='state')
    agent = InputInjester(agent)
    agent = SimpleModelRunner(agent)

    agent = AgentHead(agent)
    
    agent = apply_dp_augmentation_fns(agent,dp_augmentation_fns)

    return agent

In [ ]:
#|hide
#|eval: false
from fastcore.imports import in_colab

# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab():
    from nbdev import nbdev_export
    nbdev_export()